In [28]:
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

import warnings
warnings.filterwarnings("ignore")

In [29]:
df_ = pd.read_excel(r'C:\Users\seldabayman\OneDrive\Desktop\online_retail_II.xlsx')

In [ ]:
#Filtered InvoiceDate from 2009 to 2010 

In [30]:
# Start_end dates
start_date = "2009-01-01"
end_date = "2010-12-31"

# Datetime Format
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# 2009-2010 years filters
df = df[(df['InvoiceDate'] >= start_date) & (df['InvoiceDate'] <= end_date)]

print(df.head())

  Invoice StockCode                          Description  Quantity  \
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1  489434    79323P                   PINK CHERRY LIGHTS        12   
2  489434    79323W                  WHITE CHERRY LIGHTS        12   
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   

          InvoiceDate   Price  Customer ID         Country  Total_Price  
0 2009-12-01 07:45:00 6.95000  13085.00000  United Kingdom     83.40000  
1 2009-12-01 07:45:00 6.75000  13085.00000  United Kingdom     81.00000  
2 2009-12-01 07:45:00 6.75000  13085.00000  United Kingdom     81.00000  
3 2009-12-01 07:45:00 2.10000  13085.00000  United Kingdom    100.80000  
4 2009-12-01 07:45:00 1.25000  13085.00000  United Kingdom     30.00000  


In [31]:
df = df_.copy()
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95000,13085.00000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10000,13085.00000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25000,13085.00000,United Kingdom


In [ ]:
#Manupulating starts with 'C'(canceled) in Invioce 

In [32]:
df= df[~ df['Invoice'].str.contains('C',na=False)] 

In [ ]:
#null values

In [33]:
df.isnull().sum() 

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107560
Country             0
dtype: int64

In [34]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Quantity,515255.00000,10.95663,-9600.00000,1.00000,3.00000,10.00000,19152.00000,104.35401
InvoiceDate,515255,2010-06-28 17:41:06.558422528,2009-12-01 07:45:00,2010-03-21 13:27:00,2010-07-06 13:13:00,2010-10-15 14:27:00,2010-12-09 20:01:00,NaN
Price,515255.00000,3.95637,-53594.36000,1.25000,2.10000,4.21000,25111.09000,127.68856
Customer ID,407695.00000,15368.50411,12346.00000,13997.00000,15321.00000,16812.00000,18287.00000,1679.79570


In [35]:
df=df[df['Quantity'] >0]

In [36]:
df.dropna(inplace=True)

In [37]:
df['Total_Price'] = df['Quantity'] * df['Price']

In [38]:
df.tail()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Total_Price
525456,538171,22271,FELTCRAFT DOLL ROSIE,2,2010-12-09 20:01:00,2.95000,17530.00000,United Kingdom,5.90000
525457,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-09 20:01:00,3.75000,17530.00000,United Kingdom,3.75000
525458,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,3.75000,17530.00000,United Kingdom,3.75000
525459,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75000,17530.00000,United Kingdom,7.50000
525460,538171,21931,JUMBO STORAGE BAG SUKI,2,2010-12-09 20:01:00,1.95000,17530.00000,United Kingdom,3.90000


In [ ]:
#1. Calculated total_price,total_unit, total_transaction 

In [40]:
cltv_c = df.groupby('Customer ID').agg({'Invoice':lambda x:x.nunique(),
                                      'Quantity': lambda x:x.sum(),
                                      'Total_Price': lambda x:x.sum()})

In [41]:
cltv_c.columns= ['total_transaction','total_unit','total_price']

In [ ]:
# 2.Average Order Value (average_order_value = total_price / total_transaction)

In [42]:
cltv_c.head()

,total_transaction,total_unit,total_price
Customer ID,,,
12346.00000,11,70,372.86000
12347.00000,2,828,1323.32000
12348.00000,1,373,222.16000
12349.00000,3,993,2671.14000
12351.00000,1,261,300.93000


In [47]:
cltv_c['average_order_value'] = cltv_c['total_price'] / cltv_c['total_transaction']
cltv_c['average_order_value']

Customer ID
12346.00000    33.89636
12347.00000   661.66000
12348.00000   222.16000
12349.00000   890.38000
12351.00000   300.93000
                 ...   
18283.00000   106.96167
18284.00000   461.68000
18285.00000   427.00000
18286.00000   648.21500
18287.00000   586.42750
Name: average_order_value, Length: 4314, dtype: float64

In [ ]:
## 3.Purchase Frequency (total_transaction / total_number_of_customers)

In [48]:
cltv_c.head()
cltv_c.shape [0]
cltv_c ['purchase_frequency']= cltv_c['total_price'] / cltv_c.shape[0]

In [ ]:
#4. Repeat Rate & Churn Rate (total_transaction > 1 /total_customer)

In [51]:
repeat_rate = cltv_c [cltv_c ['total_transaction'] > 1].shape[0] / cltv_c.shape [0]
churn_rate = 1- repeat_rate

In [ ]:
# 5.Profit Margin (profit_margin =  total_price * 0.10)

In [53]:
cltv_c['profit_margin']= cltv_c['total_price'] * 0.10
cltv_c['profit_margin']

Customer ID
12346.00000    37.28600
12347.00000   132.33200
12348.00000    22.21600
12349.00000   267.11400
12351.00000    30.09300
                 ...   
18283.00000    64.17700
18284.00000    46.16800
18285.00000    42.70000
18286.00000   129.64300
18287.00000   234.57100
Name: profit_margin, Length: 4314, dtype: float64

In [ ]:
# 6.Customer Value (customer_value = average_order_value * purchase_frequency)

In [54]:
cltv_c['customer_value'] = cltv_c ['average_order_value'] * cltv_c ['purchase_frequency']

In [ ]:
# 7.Customer Lifetime Value (CLTV = (customer_value / churn_rate) x profit_margin)

In [58]:
cltv_c ["cltv"] = (cltv_c["customer_value"] / churn_rate) * cltv_c["profit_margin"]

In [59]:
cltv_c.sort_values(by='cltv', ascending=False).head()

,total_transaction,total_unit,total_price,purchase_frequency,average_order_value,profit_margin,customer_value,cltv
Customer ID,,,,,,,,
18102.00000,89,124216,349164.35000,80.93749,3923.19494,34916.43500,317533.56803,33659340445.42545
14646.00000,78,170342,248396.50000,57.57916,3184.57051,24839.65000,183364.89786,13827629546.04798
14156.00000,102,108107,196566.74000,45.56484,1927.12490,19656.67400,87809.14686,5240055122.57278
13694.00000,94,125893,131443.19000,30.46898,1398.33181,13144.31900,42605.74724,1700169965.59886
17450.00000,7,15375,52422.30000,12.15167,7488.90000,5242.23000,91002.63386,1448291882.26679


In [ ]:
#Creating Customer Segments

In [60]:
cltv_c.sort_values('cltv', ascending=False).tail()

,total_transaction,total_unit,total_price,purchase_frequency,average_order_value,profit_margin,customer_value,cltv
Customer ID,,,,,,,,
15913.00000,1,3,6.30000,0.00146,6.30000,0.63000,0.00920,0.01760
13788.00000,1,1,3.75000,0.00087,3.75000,0.37500,0.00326,0.00371
14095.00000,1,1,2.95000,0.00068,2.95000,0.29500,0.00202,0.00181
14103.00000,1,5,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
14827.00000,1,5,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


In [62]:
cltv_c['segment'] = pd.qcut (cltv_c ['cltv'], 4, labels=['D','C','B','A'])

In [64]:
cltv_c.sort_values('segment',ascending=False).head()

,total_transaction,total_unit,total_price,purchase_frequency,average_order_value,profit_margin,customer_value,cltv,segment
Customer ID,,,,,,,,,
18287.00000,4,1427,2345.71000,0.54374,586.42750,234.57100,318.86621,227074.77296,A
13473.00000,6,734,2235.12000,0.51811,372.52000,223.51200,193.00577,130965.59058,A
14733.00000,4,2727,6599.06000,1.52968,1649.76500,659.90600,2523.62036,5055826.50061,A
15270.00000,14,2489,6145.37000,1.42452,438.95500,614.53700,625.29923,1166600.05975,A
13478.00000,5,1189,2215.78000,0.51363,443.15600,221.57800,227.61618,153114.32772,A


In [65]:
cltv_c.groupby('segment').agg({'count','mean','sum'})

total_transaction                 total_unit                      \
                    count    sum     mean      count      sum       mean   
segment                                                                    
D                    1079   1524  1.41242       1079   121568  112.66728   
C                    1078   2378  2.20594       1078   315349  292.53154   
B                    1078   4304  3.99258       1078   743624  689.81818   
A                    1079  11009 10.20297       1079  4358683 4039.55792   

        total_price                          purchase_frequency             \
              count           sum       mean              count        sum   
segment                                                                      
D              1079  197855.44000  183.36927               1079   45.86357   
C              1078  527478.00200  489.31169               1078  122.27121   
B              1078 1248757.85100 1158.40246               1078  289.46635   
A              1079 6857911.98100 6355.80350               1079 1589.68752   

                average_order_value                        profit_margin  \
           mean               count          sum      mean         count   
segment                                                                    
D       0.04251                1079 154163.62792 142.87639          1079   
C       0.11342                1078 296406.50359 274.95965          1078   
B       0.26852                1078 400053.87071 371.10749          1078   
A       1.47330                1079 780703.16553 723.54325          1079   

                               customer_value                           cltv  \
                 sum      mean          count           sum       mean count   
segment                                                                        
D        19785.54400  18.33693           1079    7231.29728    6.70185  1079   
C        52747.80020  48.93117           1078   32043.64052   29.72508  1078   
B       124875.78510 115.84025           1078  100331.38504   93.07179  1078   
A       685791.19810 635.58035           1079 2051039.61952 1900.87082  1079   

                                          
                      sum           mean  
segment                                   
D            499578.98401      463.00184  
C           4939314.89901     4581.92477  
B          36767702.67274    34107.33087  
A       66263844297.76353 61412274.60404

In [ ]:
# Export as csv

In [66]:
cltv_c.to_csv ('cltv_c.csv')

In [ ]:
# All process to Functionalization

In [67]:
def create_cltv_c(dataframe, profit=0.10):

    # Prepared Data 
    dataframe = dataframe[~dataframe["Invoice"].str.contains("C", na=False)]
    dataframe = dataframe[(dataframe['Quantity'] > 0)]
    dataframe.dropna(inplace=True)
    dataframe["TotalPrice"] = dataframe["Quantity"] * dataframe["Price"]
    cltv_c = dataframe.groupby('Customer ID').agg({'Invoice': lambda x: x.nunique(),
                                                   'Quantity': lambda x: x.sum(),
                                                   'TotalPrice': lambda x: x.sum()})
    cltv_c.columns = ['total_transaction', 'total_unit', 'total_price']
    # avg_order_value
    cltv_c['avg_order_value'] = cltv_c['total_price'] / cltv_c['total_transaction']
    # purchase_frequency
    cltv_c["purchase_frequency"] = cltv_c['total_transaction'] / cltv_c.shape[0]
    # repeat rate & churn rate
    repeat_rate = cltv_c[cltv_c.total_transaction > 1].shape[0] / cltv_c.shape[0]
    churn_rate = 1 - repeat_rate
    # profit_margin
    cltv_c['profit_margin'] = cltv_c['total_price'] * profit
    # Customer Value
    cltv_c['customer_value'] = (cltv_c['avg_order_value'] * cltv_c["purchase_frequency"])
    # Customer Lifetime Value
    cltv_c['cltv'] = (cltv_c['customer_value'] / churn_rate) * cltv_c['profit_margin']
    # Segment
    cltv_c["segment"] = pd.qcut(cltv_c["cltv"], 4, labels=["D", "C", "B", "A"])

    return cltv_c


In [69]:
df= df_.copy()
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95000,13085.00000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10000,13085.00000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25000,13085.00000,United Kingdom


In [70]:
clv= create_cltv_c(df) 

In [71]:
clv

,total_transaction,total_unit,total_price,avg_order_value,purchase_frequency,profit_margin,customer_value,cltv,segment
Customer ID,,,,,,,,,
12346.00000,11,70,372.86000,33.89636,0.00255,37.28600,0.08643,9.78357,C
12347.00000,2,828,1323.32000,661.66000,0.00046,132.33200,0.30675,123.23546,B
12348.00000,1,373,222.16000,222.16000,0.00023,22.21600,0.05150,3.47326,D
12349.00000,3,993,2671.14000,890.38000,0.00070,267.11400,0.61918,502.11041,A
12351.00000,1,261,300.93000,300.93000,0.00023,30.09300,0.06976,6.37290,D
...,...,...,...,...,...,...,...,...,...
18283.00000,6,336,641.77000,106.96167,0.00139,64.17700,0.14876,28.98443,C
18284.00000,1,494,461.68000,461.68000,0.00023,46.16800,0.10702,14.99989,C
18285.00000,1,145,427.00000,427.00000,0.00023,42.70000,0.09898,12.83103,C
